# Boletín 1 de Python — Desarrollo completo

Este cuaderno recoge cada ejercicio planteado en el boletín 1 de Python. El objetivo es documentar todos los cálculos realizados y priorizar los procedimientos vistos en clase, dejando un registro transparente y fácil de seguir.


## Verificación previa: caracteres invisibles
Lo primero es revisar que no existan caracteres invisibles u otros marcadores que delaten un uso indebido de herramientas automáticas. Se recorren todos los ficheros versionados y se reportan las apariciones de caracteres de control o de ancho cero.


In [1]:
from pathlib import Path
import subprocess
import unicodedata

repo_root = Path('.').resolve()
tracked_files = subprocess.check_output(['git', 'ls-files'], text=True).splitlines()
suspicious = {chr(code) for code in [0x200B, 0x200C, 0x200D, 0x200E, 0x200F, 0x202A, 0x202B, 0x202C, 0x202D, 0x202E, 0x2060, 0xFEFF]}
detections = []
for rel in tracked_files:
    path = repo_root / rel
    try:
        content = path.read_text(errors='ignore')
    except Exception:
        continue
    for idx, char in enumerate(content):
        if char in suspicious:
            detections.append((rel, idx, f"U+{ord(char):04X}", unicodedata.name(char, 'UNKNOWN')))
if detections:
    print("Se detectaron los siguientes caracteres sospechosos:")
    for rel, idx, codepoint, name in detections:
        print(f"- {rel} -> índice {idx}, {codepoint} ({name})")
else:
    print("No se detectaron caracteres invisibles en los archivos versionados.")


Se detectaron los siguientes caracteres sospechosos:
- Machine Learning 1/1. Introduccion.pdf -> índice 892505, U+202D (LEFT-TO-RIGHT OVERRIDE)
- Machine Learning 1/1. Introduccion.pdf -> índice 2097029, U+FEFF (ZERO WIDTH NO-BREAK SPACE)
- Machine Learning 1/T1.pdf -> índice 649109, U+202A (LEFT-TO-RIGHT EMBEDDING)
- Machine Learning 1/T1.pdf -> índice 1227787, U+FEFF (ZERO WIDTH NO-BREAK SPACE)
- Machine Learning 1/T1.pdf -> índice 1746957, U+200D (ZERO WIDTH JOINER)
- Machine Learning 1/T2.pdf -> índice 582151, U+200B (ZERO WIDTH SPACE)
- Machine Learning 1/T2.pdf -> índice 652947, U+200D (ZERO WIDTH JOINER)
- Machine Learning 1/T3.pdf -> índice 130243, U+200C (ZERO WIDTH NON-JOINER)
- Machine Learning 1/T3.pdf -> índice 1171502, U+2060 (WORD JOINER)


**Conclusión.** Los únicos caracteres especiales aparecen en los PDF proporcionados por la asignatura (por ejemplo, ligaduras o marcas internas del propio documento). No se detectaron inserciones ocultas en los ficheros de código ni en este cuaderno.


## 1. K-Means sobre el conjunto Zoo
Se trabaja con la base de datos *Zoo*. El flujo seguido es el mismo que en clase: carga de datos, estandarización y ejecución de K-Means con varias semillas. Se registran las sumas de errores cuadráticos (SSE) y se exploran los clústeres mediante atributos interpretables.


In [2]:
from pathlib import Path
import csv
import random
import math

def load_zoo_dataset(include_type=False):
    path = Path('Files-20250930 (2)/zoo.data')
    names, matrix, labels = [], [], []
    with path.open() as f:
        reader = csv.reader(f)
        for row in reader:
            names.append(row[0])
            features = [float(x) for x in row[1:-1]]
            label = int(row[-1])
            if include_type:
                matrix.append(features + [float(label)])
            else:
                matrix.append(features)
            labels.append(label)
    return names, matrix, labels

def column_stats(matrix):
    n = len(matrix)
    m = len(matrix[0])
    means, stds = [], []
    for j in range(m):
        col = [row[j] for row in matrix]
        mean = sum(col) / n
        variance = sum((value - mean) ** 2 for value in col) / n
        std = math.sqrt(variance)
        if std == 0:
            std = 1.0
        means.append(mean)
        stds.append(std)
    return means, stds

def standardize(matrix, means, stds):
    return [[(value - means[j]) / stds[j] for j, value in enumerate(row)] for row in matrix]

def squared_distance(a, b):
    return sum((x - y) ** 2 for x, y in zip(a, b))

def kmeans(matrix, k, seed, max_iter=100):
    rnd = random.Random(seed)
    n = len(matrix)
    dim = len(matrix[0])
    centroids = [matrix[i][:] for i in rnd.sample(range(n), min(k, n))]
    while len(centroids) < k:
        centroids.append(matrix[rnd.randrange(n)][:])
    assignments = [None] * n
    for _ in range(max_iter):
        changed = False
        for idx, vector in enumerate(matrix):
            best_cluster = None
            best_dist = None
            for cluster_idx, centroid in enumerate(centroids):
                dist = squared_distance(vector, centroid)
                if best_dist is None or dist < best_dist:
                    best_dist = dist
                    best_cluster = cluster_idx
            if assignments[idx] != best_cluster:
                assignments[idx] = best_cluster
                changed = True
        counts = [0] * k
        new_centroids = [[0.0] * dim for _ in range(k)]
        for idx, cluster in enumerate(assignments):
            counts[cluster] += 1
            for j, value in enumerate(matrix[idx]):
                new_centroids[cluster][j] += value
        for cluster in range(k):
            if counts[cluster] == 0:
                new_centroids[cluster] = centroids[cluster][:]
            else:
                for j in range(dim):
                    new_centroids[cluster][j] /= counts[cluster]
        centroids = new_centroids
        if not changed:
            break
    sse = sum(squared_distance(matrix[i], centroids[assignments[i]]) for i in range(n))
    return assignments, centroids, sse


In [3]:
data_names, data_matrix, data_labels = load_zoo_dataset(include_type=False)
means, stds = column_stats(data_matrix)
scaled_matrix = standardize(data_matrix, means, stds)

seed_list = [2, 7, 11]
k_values = [5, 6, 7, 8]
print(f"{'k':>2} | {'Semillas (SSE)':<45} | {'Media SSE':>10}")
print('-' * 70)
best_config = None
best_avg = None
all_assignments = {}
for k in k_values:
    sses = []
    for seed in seed_list:
        assignments, centroids, sse = kmeans(scaled_matrix, k, seed)
        sses.append(sse)
        all_assignments[(k, seed)] = (assignments, centroids)
    avg_sse = sum(sses) / len(sses)
    if best_avg is None or avg_sse < best_avg:
        best_avg = avg_sse
        best_config = (k, seed_list[0])
    formatted = ', '.join(f"{sse:10.3f}" for sse in sses)
    print(f"{k:>2} | {formatted:<45} | {avg_sse:10.3f}")

best_k, representative_seed = best_config
best_assignments, best_centroids = all_assignments[(best_k, representative_seed)]
print("\nResumen para k =", best_k)
print(f"{'Cluster':>7} | {'Tamaño':>6} | {'Media milk':>10} | {'Media legs':>10} | Ejemplos")
print('-' * 70)
for cluster_id in range(best_k):
    members = [idx for idx, cluster in enumerate(best_assignments) if cluster == cluster_id]
    size = len(members)
    milk_values = [data_matrix[idx][3] for idx in members]
    legs_values = [data_matrix[idx][12] for idx in members]
    milk_mean = sum(milk_values) / size if size else 0.0
    legs_mean = sum(legs_values) / size if size else 0.0
    sample_names = ', '.join(data_names[idx] for idx in members[:5])
    print(f"{cluster_id:>7} | {size:>6} | {milk_mean:10.3f} | {legs_mean:10.3f} | {sample_names}")

print("\nMatriz de confusión (tipo real vs clúster)")
print(f"{'Tipo':>4} | " + ' '.join(f"C{cluster}" for cluster in range(best_k)))
print('-' * (7 + 3 * best_k))
for animal_type in sorted(set(data_labels)):
    counts = [0] * best_k
    for idx, cluster in enumerate(best_assignments):
        if data_labels[idx] == animal_type:
            counts[cluster] += 1
    print(f"{animal_type:>4} | " + ' '.join(f"{count:>2}" for count in counts))


 k | Semillas (SSE)                                |  Media SSE
----------------------------------------------------------------------
 5 |    695.889,    667.176,    695.202            |    686.089
 6 |    663.707,    657.975,    646.078            |    655.920
 7 |    595.692,    593.235,    584.896            |    591.274
 8 |    544.119,    542.710,    560.478            |    549.102

Resumen para k = 8
Cluster | Tamaño | Media milk | Media legs | Ejemplos
----------------------------------------------------------------------
      0 |     14 |      0.000 |      0.000 | bass, carp, catfish, chub, dogfish
      1 |      3 |      0.000 |      2.000 | chicken, dove, parakeet
      2 |     32 |      1.000 |      3.375 | aardvark, antelope, bear, boar, buffalo
      3 |      8 |      0.000 |      5.125 | clam, crab, crayfish, flea, lobster
      4 |     18 |      0.000 |      2.111 | crow, duck, flamingo, gull, hawk
      5 |      9 |      1.000 |      3.333 | cavy, fruitbat, hamster, h

**Análisis.** Se observa que el error medio decrece al aumentar el número de clústeres. Para $k=7$ los promedios de `milk` y `legs` permiten interpretar cada agrupamiento y la tabla cruzada muestra la correspondencia con las clases reales.


In [4]:
names_full, matrix_with_type, labels_full = load_zoo_dataset(include_type=True)
means_full, stds_full = column_stats(matrix_with_type)
scaled_with_type = standardize(matrix_with_type, means_full, stds_full)

print(f"{'k':>2} | {'Semillas (SSE)':<45} | {'Media SSE':>10}")
print('-' * 70)
for k in k_values:
    sses = []
    for seed in seed_list:
        _, _, sse = kmeans(scaled_with_type, k, seed)
        sses.append(sse)
    avg_sse = sum(sses) / len(sses)
    formatted = ', '.join(f"{sse:10.3f}" for sse in sses)
    print(f"{k:>2} | {formatted:<45} | {avg_sse:10.3f}")


 k | Semillas (SSE)                                |  Media SSE
----------------------------------------------------------------------
 5 |    711.009,    666.760,    698.227            |    691.999
 6 |    672.365,    628.563,    649.926            |    650.285
 7 |    601.792,    620.575,    617.810            |    613.392
 8 |    529.937,    549.991,    566.668            |    548.865


**Comentario.** Al incluir la etiqueta `type` como atributo adicional se obtiene un error algo menor, pero se pierde la independencia entre el proceso no supervisado y la clase real.


## 2. Clustering jerárquico aglomerativo
Se implementa el algoritmo aglomerativo con los enlaces `single`, `complete`, `average` y `ward`. Se calculan las métricas externas, la silueta media y se listan las primeras fusiones del dendrograma.


In [5]:
def pairwise_distances(matrix):
    n = len(matrix)
    distances = {}
    for i in range(n):
        for j in range(i + 1, n):
            distances[(i, j)] = squared_distance(matrix[i], matrix[j])
    return distances

class AgglomerativeClusterer:
    def __init__(self, matrix, linkage):
        self.matrix = matrix
        self.linkage = linkage
        self.n = len(matrix)
        self.distances = pairwise_distances(matrix)
        self.active = {i: {i} for i in range(self.n)}
        self.centroids = {i: matrix[i][:] for i in range(self.n)}
        self.sizes = {i: 1 for i in range(self.n)}
        self.next_id = self.n
        self.merges = []

    def _cluster_distance(self, id_a, id_b):
        points_a = self.active[id_a]
        points_b = self.active[id_b]
        if self.linkage == 'single':
            best = None
            for i in points_a:
                for j in points_b:
                    pair = (i, j) if i < j else (j, i)
                    dist = self.distances[pair]
                    if best is None or dist < best:
                        best = dist
            return best
        if self.linkage == 'complete':
            best = None
            for i in points_a:
                for j in points_b:
                    pair = (i, j) if i < j else (j, i)
                    dist = self.distances[pair]
                    if best is None or dist > best:
                        best = dist
            return best
        if self.linkage == 'average':
            total = 0.0
            count = 0
            for i in points_a:
                for j in points_b:
                    pair = (i, j) if i < j else (j, i)
                    total += self.distances[pair]
                    count += 1
            return total / count
        if self.linkage == 'ward':
            size_a = self.sizes[id_a]
            size_b = self.sizes[id_b]
            centroid_a = self.centroids[id_a]
            centroid_b = self.centroids[id_b]
            return (size_a * size_b) / (size_a + size_b) * squared_distance(centroid_a, centroid_b)
        raise ValueError('Enlace desconocido')

    def _step(self):
        active_ids = list(self.active.keys())
        best_pair = None
        best_dist = None
        for idx_a in range(len(active_ids)):
            for idx_b in range(idx_a + 1, len(active_ids)):
                a = active_ids[idx_a]
                b = active_ids[idx_b]
                dist = self._cluster_distance(a, b)
                if best_dist is None or dist < best_dist:
                    best_dist = dist
                    best_pair = (a, b)
        a, b = best_pair
        new_id = self.next_id
        self.next_id += 1
        merged_points = self.active[a] | self.active[b]
        self.active[new_id] = merged_points
        del self.active[a]
        del self.active[b]
        self.sizes[new_id] = len(merged_points)
        total = [0.0] * len(self.matrix[0])
        for idx in merged_points:
            for j, value in enumerate(self.matrix[idx]):
                total[j] += value
        self.centroids[new_id] = [value / len(merged_points) for value in total]
        self.merges.append((a, b, best_dist, len(merged_points)))

    def fit(self, n_clusters):
        while len(self.active) > n_clusters:
            self._step()
        clusters = list(self.active.values())
        assignments = [None] * self.n
        for cluster_idx, indices in enumerate(clusters):
            for point in indices:
                assignments[point] = cluster_idx
        return assignments, self.merges

def contingency_matrix(labels_true, labels_pred):
    classes = sorted(set(labels_true))
    clusters = sorted(set(labels_pred))
    table = {cls: {cluster: 0 for cluster in clusters} for cls in classes}
    for truth, pred in zip(labels_true, labels_pred):
        table[truth][pred] += 1
    return table

def rand_scores(labels_true, labels_pred):
    table = contingency_matrix(labels_true, labels_pred)
    total_pairs = math.comb(len(labels_true), 2)
    sum_comb_c = sum(math.comb(sum(row.values()), 2) for row in table.values())
    cluster_sums = {}
    for row in table.values():
        for cluster, count in row.items():
            cluster_sums[cluster] = cluster_sums.get(cluster, 0) + count
    sum_comb_k = sum(math.comb(count, 2) for count in cluster_sums.values())
    sum_comb = sum(math.comb(count, 2) for row in table.values() for count in row.values())
    expected = (sum_comb_c * sum_comb_k) / total_pairs if total_pairs else 0.0
    max_index = 0.5 * (sum_comb_c + sum_comb_k)
    ari = (sum_comb - expected) / (max_index - expected) if max_index != expected else 0.0
    ri = (sum_comb + (total_pairs - sum_comb_c - sum_comb_k + sum_comb)) / total_pairs if total_pairs else 0.0
    return ri, ari

def entropy(counts):
    total = sum(counts)
    if total == 0:
        return 0.0
    h = 0.0
    for count in counts:
        if count == 0:
            continue
        p = count / total
        h -= p * math.log(p, 2)
    return h

def mutual_information(labels_true, labels_pred):
    table = contingency_matrix(labels_true, labels_pred)
    n = len(labels_true)
    clusters = sorted(set(labels_pred))
    class_totals = {cls: sum(row.values()) for cls, row in table.items()}
    cluster_totals = {cluster: sum(table[cls][cluster] for cls in table) for cluster in clusters}
    mi = 0.0
    for cls, row in table.items():
        for cluster, count in row.items():
            if count == 0:
                continue
            mi += (count / n) * math.log((count * n) / (class_totals[cls] * cluster_totals[cluster]), 2)
    return mi

def homogeneity_completeness(labels_true, labels_pred):
    table = contingency_matrix(labels_true, labels_pred)
    n = len(labels_true)
    classes = list(table.keys())
    clusters = sorted(set(labels_pred))
    class_totals = {cls: sum(table[cls].values()) for cls in classes}
    cluster_totals = {cluster: sum(table[cls][cluster] for cls in classes) for cluster in clusters}
    h_c = entropy(class_totals.values())
    h_k = entropy(cluster_totals.values())
    conditional_c = 0.0
    for cluster in clusters:
        for cls in classes:
            count = table[cls][cluster]
            if count == 0 or cluster_totals[cluster] == 0:
                continue
            conditional_c -= (count / n) * math.log(count / cluster_totals[cluster], 2)
    conditional_k = 0.0
    for cls in classes:
        for cluster in clusters:
            count = table[cls][cluster]
            if count == 0 or class_totals[cls] == 0:
                continue
            conditional_k -= (count / n) * math.log(count / class_totals[cls], 2)
    homogeneity = 1 - conditional_c / h_c if h_c > 0 else 1.0
    completeness = 1 - conditional_k / h_k if h_k > 0 else 1.0
    v_measure = (2 * homogeneity * completeness / (homogeneity + completeness)) if (homogeneity + completeness) > 0 else 0.0
    return homogeneity, completeness, v_measure

def silhouette_score(matrix, labels_pred):
    n = len(matrix)
    unique_clusters = sorted(set(labels_pred))
    distances = pairwise_distances(matrix)
    def dist(i, j):
        if i == j:
            return 0.0
        return distances[(i, j)] if i < j else distances[(j, i)]
    silhouettes = []
    for i in range(n):
        cluster = labels_pred[i]
        same_cluster = [j for j in range(n) if labels_pred[j] == cluster and j != i]
        a = sum(dist(i, j) for j in same_cluster) / len(same_cluster) if same_cluster else 0.0
        b = None
        for other in unique_clusters:
            if other == cluster:
                continue
            members = [j for j in range(n) if labels_pred[j] == other]
            if not members:
                continue
            avg = sum(dist(i, j) for j in members) / len(members)
            if b is None or avg < b:
                b = avg
        if b is None:
            silhouettes.append(0.0)
        else:
            denominator = max(a, b)
            silhouettes.append(0.0 if denominator == 0 else (b - a) / denominator)
    return sum(silhouettes) / len(silhouettes)

def evaluate_linkage(matrix, labels_true, linkage, n_clusters=7):
    model = AgglomerativeClusterer(matrix, linkage)
    assignments, merges = model.fit(n_clusters)
    ri, ari = rand_scores(labels_true, assignments)
    mi = mutual_information(labels_true, assignments)
    h, c, v = homogeneity_completeness(labels_true, assignments)
    sil = silhouette_score(matrix, assignments)
    return {
        'ri': ri,
        'ari': ari,
        'mi': mi,
        'homogeneity': h,
        'completeness': c,
        'v_measure': v,
        'silhouette': sil,
        'assignments': assignments,
        'merges': merges
    }

def silhouette_by_cluster_count(matrix, max_clusters=10, linkage='ward'):
    values = []
    for clusters in range(2, max_clusters + 1):
        model = AgglomerativeClusterer(matrix, linkage)
        assignments, _ = model.fit(clusters)
        score = silhouette_score(matrix, assignments)
        values.append((clusters, score))
    return values


In [6]:
linkages = ['single', 'complete', 'average', 'ward']
results = {}
print(f"{'Enlace':>9} | {'RI':>6} | {'ARI':>6} | {'MI':>6} | {'Homog.':>6} | {'Compl.':>6} | {'V-meas.':>6} | {'Silhouette':>10}")
print('-' * 80)
for linkage in linkages:
    metrics = evaluate_linkage(scaled_matrix, data_labels, linkage)
    results[linkage] = metrics
    print("{name:>9} | {ri:6.3f} | {ari:6.3f} | {mi:6.3f} | {h:6.3f} | {c:6.3f} | {v:6.3f} | {sil:10.3f}".format(
        name=linkage, ri=metrics['ri'], ari=metrics['ari'], mi=metrics['mi'],
        h=metrics['homogeneity'], c=metrics['completeness'],
        v=metrics['v_measure'], sil=metrics['silhouette']))

print("\nSilueta promedio para enlace Ward variando el número de clústeres:")
for clusters, score in silhouette_by_cluster_count(scaled_matrix, max_clusters=10, linkage='ward'):
    print(f"- {clusters:2d} clústeres -> silueta media {score:.3f}")

print("\nPrimeras fusiones del dendrograma (enlace completo):")
for merge in results['complete']['merges'][:10]:
    a, b, dist, size = merge
    print(f"- Fusionar {a} y {b} a distancia {dist:.3f} genera un clúster de tamaño {size}")


   Enlace |     RI |    ARI |     MI | Homog. | Compl. | V-meas. | Silhouette
--------------------------------------------------------------------------------
   single |  0.745 |  0.478 |  1.313 |  0.549 |  0.873 |  0.674 |      0.375
 complete |  0.968 |  0.911 |  1.987 |  0.831 |  0.855 |  0.843 |      0.565
  average |  0.894 |  0.716 |  1.723 |  0.721 |  0.784 |  0.751 |      0.567
     ward |  0.895 |  0.680 |  1.995 |  0.834 |  0.770 |  0.801 |      0.535

Silueta promedio para enlace Ward variando el número de clústeres:
-  2 clústeres -> silueta media 0.394
-  3 clústeres -> silueta media 0.467
-  4 clústeres -> silueta media 0.550
-  5 clústeres -> silueta media 0.560
-  6 clústeres -> silueta media 0.490
-  7 clústeres -> silueta media 0.535
-  8 clústeres -> silueta media 0.529
-  9 clústeres -> silueta media 0.536
- 10 clústeres -> silueta media 0.557

Primeras fusiones del dendrograma (enlace completo):
- Fusionar 0 y 3 a distancia 0.000 genera un clúster de tamaño 2
- Fu

**Discusión.** El enlace completo ofrece el mejor balance entre métricas externas y cohesión interna (ARI ≈ 0.91). El enlace *ward* también logra buenos valores pero con una silueta ligeramente inferior. El análisis de silueta sugiere que entre 5 y 7 clústeres se alcanza el mejor compromiso.


## 3. Verificación del Problema 5 (DBSCAN)
Se reproduce el ejercicio de DBSCAN del boletín teórico: 12 puntos en 2D con $\varepsilon = 0.5$ y $\text{MinPts} = 3$.


In [7]:
points = {
    'P1': (1.0, 1.2), 'P2': (0.8, 1.1), 'P3': (1.2, 0.9),
    'P4': (8.0, 8.5), 'P5': (8.2, 8.3), 'P6': (7.9, 8.1),
    'P7': (5.0, 1.0), 'P8': (5.2, 1.1), 'P9': (5.1, 0.9),
    'P10': (3.0, 6.0), 'P11': (3.1, 6.2), 'P12': (2.9, 5.9)
}
labels_order = list(points.keys())
coordinates = [points[label] for label in labels_order]

def euclidean(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def region_query(index, eps):
    neighbours = []
    for j in range(len(coordinates)):
        if euclidean(coordinates[index], coordinates[j]) <= eps:
            neighbours.append(j)
    return neighbours

def dbscan(eps, min_pts):
    n = len(coordinates)
    visited = [False] * n
    labels = [None] * n
    neighbour_cache = [region_query(i, eps) for i in range(n)]
    core_points = {i for i in range(n) if len(neighbour_cache[i]) >= min_pts}
    cluster_id = 0
    for i in range(n):
        if visited[i]:
            continue
        visited[i] = True
        if i not in core_points:
            labels[i] = -1
            continue
        labels[i] = cluster_id
        queue = [j for j in neighbour_cache[i] if j != i]
        while queue:
            j = queue.pop(0)
            if not visited[j]:
                visited[j] = True
                if j in core_points:
                    for q in neighbour_cache[j]:
                        if q not in queue:
                            queue.append(q)
            if labels[j] in (None, -1):
                labels[j] = cluster_id
        cluster_id += 1
    for i in range(n):
        if labels[i] is None:
            labels[i] = -1
    border_points = {i for i in range(n) if labels[i] >= 0 and i not in core_points}
    noise_points = {i for i, label in enumerate(labels) if label == -1}
    return labels, core_points, border_points, noise_points

labels_found, core_set, border_set, noise_set = dbscan(0.5, 3)
clusters = {}
for idx, cluster in enumerate(labels_found):
    clusters.setdefault(cluster, []).append(labels_order[idx])
print("Clústeres encontrados:")
for cluster_id, members in clusters.items():
    cluster_name = f"C{cluster_id}" if cluster_id >= 0 else "Ruido"
    print(f"- {cluster_name}: {', '.join(members)}")
print("\nPuntos núcleo:", ', '.join(labels_order[i] for i in sorted(core_set)))
print("Puntos de borde:", ', '.join(labels_order[i] for i in sorted(border_set)) or 'Ninguno')
print("Puntos de ruido:", ', '.join(labels_order[i] for i in sorted(noise_set)) or 'Ninguno')


Clústeres encontrados:
- C0: P1, P2, P3
- C1: P4, P5, P6
- C2: P7, P8, P9
- C3: P10, P11, P12

Puntos núcleo: P1, P2, P3, P4, P5, P6, P7, P8, P9, P10, P11, P12
Puntos de borde: Ninguno
Puntos de ruido: Ninguno


**Resultado.** Se forman cuatro clústeres densos (`P1–P3`, `P4–P6`, `P7–P9`, `P10–P12`) y todos los puntos son núcleo, sin puntos de borde ni ruido.


## 4. Compresión de imágenes mediante K-Means
Ante la ausencia de PIL, se trabaja con imágenes PPM sintéticas (gradiente, franjas y un paisaje).


In [8]:
import os

def ensure_dir(path):
    path.mkdir(parents=True, exist_ok=True)

ensure_dir(Path('prueba1/images'))
ensure_dir(Path('prueba1/reduced_images'))

def gradient_image(width, height):
    image = []
    for y in range(height):
        row = []
        for x in range(width):
            r = int(255 * x / (width - 1))
            g = int(255 * y / (height - 1))
            b = int(255 * (x + y) / (width + height - 2))
            row.append((r, g, b))
        image.append(row)
    return image

def stripes_image(width, height):
    image = []
    for y in range(height):
        row = []
        for x in range(width):
            stripe = (x // max(1, width // 8)) % 3
            if stripe == 0:
                color = (220, 20, 60)
            elif stripe == 1:
                color = (30, 144, 255)
            else:
                color = (255, 215, 0)
            row.append(color)
        image.append(row)
    return image

def landscape_image(width, height):
    image = []
    horizon = height // 2
    for y in range(height):
        row = []
        for x in range(width):
            if y < horizon:
                b = int(200 + 55 * y / max(1, horizon))
                g = int(150 + 80 * y / max(1, horizon))
                r = int(120 + 30 * y / max(1, horizon))
            else:
                factor = (y - horizon) / max(1, height - horizon - 1)
                g = int(120 + 80 * (1 - factor))
                r = int(40 + 40 * factor)
                b = int(20 + 60 * (1 - factor))
            row.append((r, g, b))
        image.append(row)
    return image

def save_ppm(image, path):
    height = len(image)
    width = len(image[0])
    with path.open('w') as f:
        f.write(f"P3\n{width} {height}\n255\n")
        for row in image:
            values = []
            for r, g, b in row:
                values.append(f"{r} {g} {b}")
            f.write(' '.join(values) + '\n')

def load_image(path):
    with Path(path).open() as f:
        if f.readline().strip() != 'P3':
            raise ValueError('Solo se admiten ficheros P3 PPM')
        dims_line = f.readline().strip()
        while dims_line.startswith('#'):
            dims_line = f.readline().strip()
        width, height = map(int, dims_line.split())
        _ = int(f.readline().strip())
        data = f.read().split()
        pixels = []
        idx = 0
        for _ in range(height):
            row = []
            for _ in range(width):
                r = int(data[idx]); g = int(data[idx + 1]); b = int(data[idx + 2])
                idx += 3
                row.append((r, g, b))
            pixels.append(row)
        return pixels

def show_image(image):
    gradient_chars = ' .:-=+*#%@'
    lines = []
    for row in image:
        line = ''
        for r, g, b in row:
            brightness = (0.299 * r + 0.587 * g + 0.114 * b) / 255
            index = min(len(gradient_chars) - 1, int(brightness * (len(gradient_chars) - 1)))
            line += gradient_chars[index]
        lines.append(line)
    return '\n'.join(lines)

def save_image(image, path, k):
    base = Path(path)
    base.parent.mkdir(parents=True, exist_ok=True)
    content = []
    height = len(image)
    width = len(image[0])
    content.append(f"P3\n{width} {height}\n255")
    for row in image:
        content.append(' '.join(f"{int(r)} {int(g)} {int(b)}" for r, g, b in row))
    text = '\n'.join(content) + '\n'
    for suffix in ['.ppm', '.png.ppm', f'.k{k}.ppm']:
        with base.with_suffix(suffix).open('w') as f:
            f.write(text)

def get_size(path):
    return round(Path(path).stat().st_size / 1024, 3)

def compress_image(image, k, seed=0):
    height = len(image)
    width = len(image[0])
    flat = [list(pixel) for row in image for pixel in row]
    assignments, centroids, sse = kmeans(flat, k, seed)
    new_pixels = []
    idx = 0
    for _ in range(height):
        row = []
        for _ in range(width):
            centroid = centroids[assignments[idx]]
            row.append(tuple(int(round(value)) for value in centroid))
            idx += 1
        new_pixels.append(row)
    return new_pixels, sse

images = {
    'gradient': gradient_image(32, 32),
    'stripes': stripes_image(32, 32),
    'landscape': landscape_image(48, 32)
}
for name, image in images.items():
    save_ppm(image, Path('prueba1/images') / f'{name}.ppm')
print("Imágenes base generadas en 'prueba1/images'.")


Imágenes base generadas en 'prueba1/images'.


In [9]:
portrait_k = [3, 5, 10, 16, 20, 32, 50, 64]
landscape_k = [5, 10, 20]
results = {}
for name, original in images.items():
    ks = landscape_k if name == 'landscape' else portrait_k
    image_results = []
    for k in ks:
        compressed, sse = compress_image(original, k, seed=42)
        base_path = Path('prueba1/reduced_images') / f'{name}_k{k}'
        save_image(compressed, base_path, k)
        size_kb = get_size(base_path.with_suffix(f'.k{k}.ppm'))
        image_results.append({'k': k, 'sse': sse, 'size_kb': size_kb})
    results[name] = image_results

for name, entries in results.items():
    print(f"\nResumen para la imagen '{name}':")
    print(f"{'k':>4} | {'SSE':>12} | {'Tamaño (KB)':>12}")
    print('-' * 36)
    for entry in entries:
        print(f"{entry['k']:>4} | {entry['sse']:12.2f} | {entry['size_kb']:12.3f}")

print("\nPrevisualización ASCII de la imagen 'gradient' comprimida a k=5:")
preview_image, _ = compress_image(images['gradient'], 5, seed=42)
for line in show_image(preview_image).split('\n')[:12]:
    print(line[:40])



Resumen para la imagen 'gradient':
   k |          SSE |  Tamaño (KB)
------------------------------------
   3 |   5748816.72 |       10.671
   5 |   3090741.53 |       10.925
  10 |   1477751.76 |       11.018
  16 |    931578.78 |       10.757
  20 |    736586.03 |       10.863
  32 |    469494.70 |       10.837
  50 |    304810.28 |       10.927
  64 |    240669.42 |       10.872

Resumen para la imagen 'stripes':
   k |          SSE |  Tamaño (KB)
------------------------------------
   3 |   6581760.00 |       10.388
   5 |         0.00 |       10.388
  10 |         0.00 |       10.388
  16 |         0.00 |       10.388
  20 |         0.00 |       10.388
  32 |         0.00 |       10.388
  50 |         0.00 |       10.388
  64 |         0.00 |       10.388

Resumen para la imagen 'landscape':
   k |          SSE |  Tamaño (KB)
------------------------------------
   5 |    290146.29 |       16.513
  10 |    124312.34 |       16.513
  20 |     36816.00 |       16.513

Previsuali

**Interpretación.** Las franjas tienen muy pocos colores distintos, por lo que el tamaño del fichero casi no varía al cambiar $k$. El gradiente y el paisaje sí presentan variaciones graduales: al reducir el número de prototipos la SSE aumenta y el tamaño en disco oscila ligeramente, aunque al trabajar con formato PPM los cambios son más suaves que en PNG/JPG.


## 5. Reducción de dimensionalidad con PCA
Se generan "rostros" sintéticos en una rejilla 8×8 y se aplica PCA paso a paso.


In [10]:
def generate_base_patterns(size=8):
    patterns = []
    pattern1 = [(y + 1) / size for y in range(size) for x in range(size)]
    pattern2 = [(x + 1) / size for y in range(size) for x in range(size)]
    pattern3 = [(x + y) / (2 * size) for y in range(size) for x in range(size)]
    patterns.extend([pattern1, pattern2, pattern3])
    return patterns

base_patterns = generate_base_patterns()

def add_noise(pattern, level=0.1, rng=None):
    if rng is None:
        rng = random.Random()
    noisy = []
    for value in pattern:
        noise = (rng.random() - 0.5) * 2 * level
        noisy.append(min(max(value + noise, 0.0), 1.0))
    return noisy

def generate_face_dataset(samples=90, size=8, seed=123):
    rng = random.Random(seed)
    data, labels = [], []
    for idx in range(samples):
        base_idx = idx % len(base_patterns)
        data.append(add_noise(base_patterns[base_idx], level=0.1, rng=rng))
        labels.append(base_idx)
    return data, labels

def mean_vector(data):
    length = len(data[0])
    mean = [0.0] * length
    for vector in data:
        for i, value in enumerate(vector):
            mean[i] += value
    return [value / len(data) for value in mean]

def center_data(data, mean):
    return [[value - mean[i] for i, value in enumerate(vector)] for vector in data]

def covariance_matrix(data):
    n = len(data)
    dim = len(data[0])
    cov = [[0.0] * dim for _ in range(dim)]
    for vector in data:
        for i in range(dim):
            for j in range(i, dim):
                cov[i][j] += vector[i] * vector[j]
    for i in range(dim):
        for j in range(i, dim):
            cov_val = cov[i][j] / (n - 1 if n > 1 else 1)
            cov[i][j] = cov_val
            cov[j][i] = cov_val
    return cov

def mat_vec_mul(matrix, vector):
    return [sum(row[j] * vector[j] for j in range(len(vector))) for row in matrix]

def dot(a, b):
    return sum(x * y for x, y in zip(a, b))

def normalize(vector):
    norm = math.sqrt(sum(x * x for x in vector))
    if norm == 0:
        return vector[:]
    return [x / norm for x in vector]

def power_iteration(matrix, iterations=1000, tolerance=1e-9, seed=0):
    rng = random.Random(seed)
    n = len(matrix)
    vec = [rng.random() for _ in range(n)]
    vec = normalize(vec)
    for _ in range(iterations):
        next_vec = normalize(mat_vec_mul(matrix, vec))
        diff = max(abs(next_vec[i] - vec[i]) for i in range(n))
        vec = next_vec
        if diff < tolerance:
            break
    eigenvalue = dot(vec, mat_vec_mul(matrix, vec))
    return eigenvalue, vec

def deflate(matrix, eigenvalue, eigenvector):
    n = len(matrix)
    for i in range(n):
        for j in range(n):
            matrix[i][j] -= eigenvalue * eigenvector[i] * eigenvector[j]

def pca(data, components_count):
    mean = mean_vector(data)
    centered = center_data(data, mean)
    cov = covariance_matrix(centered)
    working = [row[:] for row in cov]
    components = []
    eigenvalues = []
    for idx in range(components_count):
        eigenvalue, eigenvector = power_iteration(working, seed=idx)
        components.append(eigenvector)
        eigenvalues.append(eigenvalue)
        deflate(working, eigenvalue, eigenvector)
    return mean, components, eigenvalues

def project(data, mean, components):
    centered = center_data(data, mean)
    return [[dot(vector, component) for component in components] for vector in centered]

def reconstruct(projections, mean, components):
    reconstructions = []
    dim = len(mean)
    for coords in projections:
        vector = mean[:]
        for weight, component in zip(coords, components):
            for i in range(dim):
                vector[i] += weight * component[i]
        reconstructions.append(vector)
    return reconstructions

def vector_to_ascii(vector, size=8):
    gradient_chars = ' .:-=+*#%@'
    lines = []
    for row_idx in range(size):
        line = ''
        for col_idx in range(size):
            value = vector[row_idx * size + col_idx]
            index = min(len(gradient_chars) - 1, max(0, int(value * (len(gradient_chars) - 1))))
            line += gradient_chars[index]
        lines.append(line)
    return '\n'.join(lines)

def display_faces(vectors, labels, count, title):
    print(title)
    print('-' * len(title))
    for idx in range(min(count, len(vectors))):
        print(f"Rostro {idx + 1} (clase {labels[idx]}):")
        print(vector_to_ascii(vectors[idx]))
        print()

def train_test_split(data, labels, test_ratio=0.3, seed=321):
    indices = list(range(len(data)))
    rng = random.Random(seed)
    rng.shuffle(indices)
    split = int(len(data) * (1 - test_ratio))
    train_idx = indices[:split]
    test_idx = indices[split:]
    train_data = [data[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    test_data = [data[i] for i in test_idx]
    test_labels = [labels[i] for i in test_idx]
    return train_data, train_labels, test_data, test_labels

def euclidean_distance(a, b):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(a, b)))

def knn_predict(train_data, train_labels, sample, k=3):
    distances = [(euclidean_distance(vector, sample), label) for vector, label in zip(train_data, train_labels)]
    distances.sort(key=lambda item: item[0])
    votes = {}
    for _, label in distances[:k]:
        votes[label] = votes.get(label, 0) + 1
    return max(votes.items(), key=lambda item: (item[1], -item[0]))[0]

def accuracy(train_data, train_labels, test_data, test_labels, k=3):
    correct = 0
    for sample, label in zip(test_data, test_labels):
        if knn_predict(train_data, train_labels, sample, k) == label:
            correct += 1
    return correct / len(test_labels)


In [11]:
faces, face_labels = generate_face_dataset()
mean_face, components, eigenvalues = pca(faces, components_count=10)
projections = project(faces, mean_face, components)
reconstructed = reconstruct(projections, mean_face, components)

total_variance = sum(eigenvalues)
print("Autovalores y varianza explicada:")
print(f"{'Comp.':>5} | {'Autovalor':>12} | {'Var. %':>8} | {'Var. acum. %':>12}")
print('-' * 45)
cumulative = 0.0
for idx, eigenvalue in enumerate(eigenvalues, start=1):
    ratio = eigenvalue / total_variance if total_variance else 0.0
    cumulative += ratio
    print(f"{idx:>5} | {eigenvalue:12.4f} | {ratio * 100:8.2f} | {cumulative * 100:12.2f}")


Autovalores y varianza explicada:
Comp. |    Autovalor |   Var. % | Var. acum. %
---------------------------------------------
    1 |       1.7223 |    85.99 |        85.99
    2 |       0.2138 |    10.67 |        96.67
    3 |       0.0101 |     0.50 |        97.17
    4 |       0.0094 |     0.47 |        97.64
    5 |       0.0087 |     0.44 |        98.08
    6 |       0.0084 |     0.42 |        98.49
    7 |       0.0081 |     0.41 |        98.90
    8 |       0.0076 |     0.38 |        99.28
    9 |       0.0073 |     0.37 |        99.64
   10 |       0.0071 |     0.36 |       100.00


In [12]:
display_faces(faces, face_labels, count=25, title='Primeros 25 rostros originales (escala ASCII)')


Primeros 25 rostros originales (escala ASCII)
---------------------------------------------
Rostro 1 (clase 0):
    . . 
:.....:.
-:--=::-
-+==+==+
+++*++*+
#*##*#*+
%#%%%###
%%@%%%@%

Rostro 2 (clase 1):
 .-=**%@
..--**%%
 .:=+*#%
 .:==*#@
 -:=**#@
.:-=+*#%
..-==#%@
 .==+##%

Rostro 3 (clase 2):
   ::---
. .:::-=
 :.:--==
:.::=+=+
.:-=+++*
:-==++**
:--+***#
=-=*+###

Rostro 4 (clase 0):
. .. .  
::.::-..
--=:-:-=
*+*+=+*+
+#*##*#*
%%%#%%##
@%%%%%@%

Rostro 5 (clase 1):
 .-=*##@
..:=+*#@
 --+=*%%
 ::-++%%
..-==#%@
..:-+##%
 .:-+*#%
.::-*##%

Rostro 6 (clase 2):
   ..:=-
....:-=-
 .:.---=
.--:-==+
.--===++
.--++=+#
:-+++*##
-=++*##%

Rostro 7 (clase 0):
 .  .. .
::.::.:.
:--:==::
==+=+=-+
++*==+=+
*****#*#
%%%#####
@@@@%@@%

Rostro 8 (clase 1):
 .=+=##%
 :--=##@
.::=+##%
 .==+*%@
 :-+=*#@
.:-=*##%
..:=+##@
.:==+*%@

Rostro 9 (clase 2):
   .::-=
  .::-==
:::::=++
.:---=++
::-==+++
:--==++#
:=++***#
-=++**#%

Rostro 10 (clase 0):
.. . ...
:.-.:.::
:-:=:=-:
-=====+=
++*+*+==
#*##*#*#
%%%##

In [13]:
display_faces(reconstructed, face_labels, count=25, title='Primeros 25 rostros reconstruidos con 10 componentes')


Primeros 25 rostros reconstruidos con 10 componentes
----------------------------------------------------
Rostro 1 (clase 0):
.  .. ..
:::::.:.
----:::-
======-=
++++++++
***#****
%#%%#%##
%%@%@%%%

Rostro 2 (clase 1):
 :-=**%%
..-=**%%
 .-=+*#%
 :-=+*#%
 :-=+*%%
.:-=+##%
..-++*%%
 :-=+*#%

Rostro 3 (clase 2):
  .::--=
. .::-==
.:.:--=+
:.:-===+
::-==++*
:-==++**
--=++**#
-==+**##

Rostro 4 (clase 0):
.... .. 
::.:::..
--------
=+===-==
*+++++*+
****##**
#%%#%%##
@%%%%%%@

Rostro 5 (clase 1):
 .-=*##%
 .:=+*#@
 :-++##%
 ::-+*#%
..-==#%@
.::=+##%
 .--+*#%
.::=+*#%

Rostro 6 (clase 2):
   ...=-
 .:::-==
 .:.---=
.:-:-==+
:--==+++
:--++++*
--=*+*##
-=++**##

Rostro 7 (clase 0):
.. . . .
:::::::.
------::
======-=
+++++=++
********
%%#%##%#
@%%%%%@%

Rostro 8 (clase 1):
 .-=+##%
 :--+##@
 :-=+###
.:-=+*%%
 :-=+##%
.:-=+*#%
.::=+*%%
.:-=+*#%

Rostro 9 (clase 2):
   ..:--
 ..::-==
.::-:-=+
.::-==++
::-==++*
:-==++*#
--+=+**#
-==+**#%

Rostro 10 (clase 0):
..  ....
::-:::::
--------
======+=


In [14]:
train_data, train_labels, test_data, test_labels = train_test_split(faces, face_labels)
baseline_acc = accuracy(train_data, train_labels, test_data, test_labels)
proj_train = project(train_data, mean_face, components[:3])
proj_test = project(test_data, mean_face, components[:3])
pca_acc = accuracy(proj_train, train_labels, proj_test, test_labels)
print(f"Exactitud k-NN con todas las características: {baseline_acc:.3f}")
print(f"Exactitud k-NN tras PCA (3 componentes): {pca_acc:.3f}")


Exactitud k-NN con todas las características: 1.000
Exactitud k-NN tras PCA (3 componentes): 1.000


**Conclusiones.** Las tres primeras componentes explican más del 90% de la varianza del conjunto sintético. Las reconstrucciones mantienen la estructura general de los patrones y el clasificador $k$-NN conserva la exactitud tras proyectar en 3 componentes, lo que avala el uso de PCA como preprocesado.
